# LSTM for Time Series Forecasting

In [ ]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy

In [ ]:
# date-time parsing function for loading the dataset
def parser(x):
	return datetime.strptime(x, '%Y-%m')

# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
	return model

# make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]

In [3]:
# load dataset
series = read_csv('monthly-sunspot-number-zurich-17.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

In [ ]:
series.head()

In [4]:
# transform data to be stationary
raw_values = series.values
diff_values = difference(raw_values, 1)

In [5]:
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

In [ ]:
print(supervised_values)

In [ ]:
supervised_values.shape

In [6]:
# split data into train and test-sets
train, test = supervised_values[0:-2184], supervised_values[-2184:]

# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

In [7]:
# fit the model
lstm_model = fit_lstm(train_scaled, 1, 1500, 1)
# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)

# walk-forward validation on the test data
predictions = list()

In [8]:
for i in range(len(test_scaled)):
	# make one-step forecast
	X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
	yhat = forecast_lstm(lstm_model, 1, X)
	# invert scaling
	yhat = invert_scale(scaler, X, yhat)
	# invert differencing
	yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
	# store forecast
	predictions.append(yhat)
	expected = raw_values[len(train) + i + 1]
	print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

Month=1, Predicted=45.563753, Expected=47.800000
Month=2, Predicted=46.719158, Expected=47.000000
Month=3, Predicted=46.861858, Expected=40.800000
Month=4, Predicted=42.396557, Expected=42.000000
Month=5, Predicted=42.897241, Expected=44.000000
Month=6, Predicted=44.331173, Expected=46.000000
Month=7, Predicted=46.022888, Expected=48.000000
Month=8, Predicted=47.854623, Expected=50.000000
Month=9, Predicted=49.762807, Expected=51.800000
Month=10, Predicted=51.560918, Expected=38.500000
Month=11, Predicted=42.876402, Expected=34.500000
Month=12, Predicted=38.082956, Expected=50.000000
Month=13, Predicted=46.628398, Expected=50.000000
Month=14, Predicted=48.402402, Expected=50.800000
Month=15, Predicted=50.041209, Expected=29.500000
Month=16, Predicted=36.304988, Expected=25.000000
Month=17, Predicted=30.116264, Expected=44.300000
Month=18, Predicted=39.450031, Expected=36.000000
Month=19, Predicted=36.514870, Expected=48.300000
Month=20, Predicted=45.109258, Expected=34.100000
Month=21,

Month=293, Predicted=25.426377, Expected=32.400000
Month=294, Predicted=31.231715, Expected=37.100000
Month=295, Predicted=35.598201, Expected=52.500000
Month=296, Predicted=46.889200, Expected=39.600000
Month=297, Predicted=41.451827, Expected=18.900000
Month=298, Predicted=26.749794, Expected=50.600000
Month=299, Predicted=37.335401, Expected=39.500000
Month=300, Predicted=37.346516, Expected=68.100000
Month=301, Predicted=54.735620, Expected=34.600000
Month=302, Predicted=38.483902, Expected=47.400000
Month=303, Predicted=45.871093, Expected=57.800000
Month=304, Predicted=54.197864, Expected=46.000000
Month=305, Predicted=48.353551, Expected=56.300000
Month=306, Predicted=54.859035, Expected=56.700000
Month=307, Predicted=56.102719, Expected=42.900000
Month=308, Predicted=47.298120, Expected=53.700000
Month=309, Predicted=53.063120, Expected=49.600000
Month=310, Predicted=50.295216, Expected=57.200000
Month=311, Predicted=55.757998, Expected=48.200000
Month=312, Predicted=50.424300,

Month=582, Predicted=61.404918, Expected=70.000000
Month=583, Predicted=67.597089, Expected=39.100000
Month=584, Predicted=47.809642, Expected=61.600000
Month=585, Predicted=56.479589, Expected=86.200000
Month=586, Predicted=73.238673, Expected=71.000000
Month=587, Predicted=70.261014, Expected=54.800000
Month=588, Predicted=59.970471, Expected=60.000000
Month=589, Predicted=61.838489, Expected=75.500000
Month=590, Predicted=71.366937, Expected=105.400000
Month=591, Predicted=89.525410, Expected=64.600000
Month=592, Predicted=68.875187, Expected=56.500000
Month=593, Predicted=60.706147, Expected=62.600000
Month=594, Predicted=63.470162, Expected=63.200000
Month=595, Predicted=63.839573, Expected=36.100000
Month=596, Predicted=45.284023, Expected=57.400000
Month=597, Predicted=53.251436, Expected=67.900000
Month=598, Predicted=62.696633, Expected=62.500000
Month=599, Predicted=61.716305, Expected=50.900000
Month=600, Predicted=54.409419, Expected=71.400000
Month=601, Predicted=65.305510

Month=850, Predicted=108.030643, Expected=103.500000
Month=851, Predicted=104.303465, Expected=112.000000
Month=852, Predicted=110.461635, Expected=83.900000
Month=853, Predicted=92.291625, Expected=86.700000
Month=854, Predicted=91.090577, Expected=107.000000
Month=855, Predicted=101.288432, Expected=98.300000
Month=856, Predicted=98.601691, Expected=76.200000
Month=857, Predicted=83.753495, Expected=47.900000
Month=858, Predicted=60.824901, Expected=44.800000
Month=859, Predicted=53.255659, Expected=66.900000
Month=860, Predicted=61.557017, Expected=68.200000
Month=861, Predicted=65.203465, Expected=47.500000
Month=862, Predicted=53.063187, Expected=47.400000
Month=863, Predicted=50.932449, Expected=55.400000
Month=864, Predicted=55.324466, Expected=49.200000
Month=865, Predicted=50.821626, Expected=60.800000
Month=866, Predicted=58.314921, Expected=64.200000
Month=867, Predicted=62.313756, Expected=46.400000
Month=868, Predicted=51.574340, Expected=32.000000
Month=869, Predicted=39.

Month=1153, Predicted=24.236889, Expected=30.200000
Month=1154, Predicted=26.919382, Expected=36.400000
Month=1155, Predicted=33.405684, Expected=38.300000
Month=1156, Predicted=36.539297, Expected=14.500000
Month=1157, Predicted=21.572966, Expected=25.800000
Month=1158, Predicted=26.388367, Expected=22.300000
Month=1159, Predicted=23.587848, Expected=9.000000
Month=1160, Predicted=14.049412, Expected=31.400000
Month=1161, Predicted=24.657888, Expected=34.800000
Month=1162, Predicted=30.568550, Expected=34.400000
Month=1163, Predicted=32.447673, Expected=30.900000
Month=1164, Predicted=30.759011, Expected=12.600000
Month=1165, Predicted=18.744362, Expected=19.500000
Month=1166, Predicted=21.301130, Expected=9.200000
Month=1167, Predicted=13.311554, Expected=18.100000
Month=1168, Predicted=17.987585, Expected=14.200000
Month=1169, Predicted=15.161504, Expected=7.700000
Month=1170, Predicted=9.867964, Expected=20.500000
Month=1171, Predicted=17.744330, Expected=13.500000
Month=1172, Pred

Month=1452, Predicted=7.857133, Expected=17.500000
Month=1453, Predicted=15.072549, Expected=4.500000
Month=1454, Predicted=7.800678, Expected=1.500000
Month=1455, Predicted=4.290687, Expected=3.300000
Month=1456, Predicted=4.713264, Expected=6.100000
Month=1457, Predicted=6.509363, Expected=3.200000
Month=1458, Predicted=4.294100, Expected=9.100000
Month=1459, Predicted=8.431669, Expected=3.500000
Month=1460, Predicted=4.622719, Expected=0.500000
Month=1461, Predicted=2.010605, Expected=13.200000
Month=1462, Predicted=10.195702, Expected=11.600000
Month=1463, Predicted=10.535856, Expected=10.000000
Month=1464, Predicted=10.024299, Expected=2.800000
Month=1465, Predicted=4.892273, Expected=0.500000
Month=1466, Predicted=2.436114, Expected=5.100000
Month=1467, Predicted=5.278818, Expected=1.800000
Month=1468, Predicted=2.831407, Expected=11.300000
Month=1469, Predicted=9.358956, Expected=20.800000
Month=1470, Predicted=17.352737, Expected=24.000000
Month=1471, Predicted=21.672477, Expec

Month=1763, Predicted=140.087309, Expected=95.800000
Month=1764, Predicted=110.066589, Expected=138.000000
Month=1765, Predicted=114.867655, Expected=119.100000
Month=1766, Predicted=114.658278, Expected=182.300000
Month=1767, Predicted=139.343935, Expected=157.500000
Month=1768, Predicted=145.105393, Expected=147.000000
Month=1769, Predicted=145.035702, Expected=106.200000
Month=1770, Predicted=117.573524, Expected=121.700000
Month=1771, Predicted=122.194497, Expected=125.800000
Month=1772, Predicted=125.225410, Expected=123.800000
Month=1773, Predicted=124.173848, Expected=145.300000
Month=1774, Predicted=137.161035, Expected=131.600000
Month=1775, Predicted=132.586015, Expected=143.500000
Month=1776, Predicted=140.825097, Expected=117.600000
Month=1777, Predicted=124.828233, Expected=101.600000
Month=1778, Predicted=110.290232, Expected=94.800000
Month=1779, Predicted=101.371173, Expected=109.700000
Month=1780, Predicted=107.978985, Expected=113.400000
Month=1781, Predicted=111.7843

Month=2040, Predicted=60.988589, Expected=82.200000
Month=2041, Predicted=74.352550, Expected=61.500000
Month=2042, Predicted=64.766718, Expected=88.400000
Month=2043, Predicted=78.124518, Expected=80.100000
Month=2044, Predicted=78.303982, Expected=63.200000
Month=2045, Predicted=68.121648, Expected=80.500000
Month=2046, Predicted=77.019521, Expected=88.000000
Month=2047, Predicted=84.326700, Expected=76.500000
Month=2048, Predicted=78.714064, Expected=76.800000
Month=2049, Predicted=78.311008, Expected=64.000000
Month=2050, Predicted=68.960234, Expected=61.300000
Month=2051, Predicted=64.976716, Expected=41.600000
Month=2052, Predicted=49.978397, Expected=45.300000
Month=2053, Predicted=49.338660, Expected=43.400000
Month=2054, Predicted=46.370579, Expected=42.900000
Month=2055, Predicted=44.963265, Expected=46.000000
Month=2056, Predicted=46.679919, Expected=57.700000
Month=2057, Predicted=54.748862, Expected=42.400000
Month=2058, Predicted=46.261734, Expected=39.500000
Month=2059, 

In [ ]:
# report performance
rmse = sqrt(mean_squared_error(raw_values[-2184:], predictions))
print('Test RMSE: %.3f' % rmse)

In [ ]:
# line plot of observed vs predicted
pyplot.plot(raw_values[-2184:])
pyplot.plot(predictions)
pyplot.show()

In [ ]:
raw_values.shape

In [ ]:
print(len(predictions))